# Introduction to Theano

Credits: Forked from [summerschool2015](https://github.com/mila-udem/summerschool2015) by mila-udem

## Slides

Refer to the associated [Introduction to Theano slides](http://nbviewer.ipython.org/github/donnemartin/data-science-ipython-notebooks/blob/master/deep-learning/theano-tutorial/intro_theano/intro_theano.pdf) and use this notebook for hands-on practice of the concepts.

## Basic usage

### Defining an expression

In [1]:
import theano
from theano import tensor as T
x = T.vector('x')
W = T.matrix('W')
b = T.vector('b')

ERROR (theano.sandbox.cuda): ERROR: Not using GPU. Initialisation of device gpu failed:
initCnmem: cnmemInit call failed! Reason=CNMEM_STATUS_OUT_OF_MEMORY. numdev=1



RuntimeError: Cuda error: kernel_reduce_ccontig_node_97496c4d3cf9a06dc4082cc141f918d2_0: out of memory. (grid: 1 x 1; block: 256 x 1 x 1)

Apply node that caused the error: GpuCAReduce{add}{1}(<CudaNdarrayType(float32, vector)>)
Toposort index: 0
Inputs types: [CudaNdarrayType(float32, vector)]
Inputs shapes: [(10000,)]
Inputs strides: [(1,)]
Inputs values: ['not shown']
Outputs clients: [[HostFromGpu(GpuCAReduce{add}{1}.0)]]

HINT: Re-running with most Theano optimization disabled could give you a back-trace of when this node was created. This can be done with by setting the Theano flag 'optimizer=fast_compile'. If that does not work, Theano optimizations can be disabled with 'optimizer=None'.
HINT: Use the Theano flag 'exception_verbosity=high' for a debugprint and storage map footprint of this apply node.

In [ ]:
dot = T.dot(x, W)
out = T.nnet.sigmoid(dot + b)

### Graph visualization

In [ ]:
from theano.printing import debugprint
debugprint(dot)

In [ ]:
debugprint(out)

### Compiling a Theano function

In [ ]:
f = theano.function(inputs=[x, W], outputs=dot,allow_input_downcast=True)
g = theano.function([x, W, b], out,allow_input_downcast=True)
h = theano.function([x, W, b], [dot, out],allow_input_downcast=True)
i = theano.function([x, W, b], [dot + b, out],allow_input_downcast=True)

### Graph visualization

In [ ]:
debugprint(f)

In [ ]:
debugprint(g)

In [ ]:
from theano.printing import pydotprint
pydotprint(f, outfile='pydotprint_f.png')

In [ ]:
from IPython.display import Image
Image('pydotprint_f.png', width=1000)

In [ ]:
pydotprint(g, outfile='pydotprint_g.png')
Image('pydotprint_g.png', width=1000)

In [ ]:
pydotprint(h, outfile='pydotprint_h.png')
Image('pydotprint_h.png', width=1000)

### Executing a Theano function

In [ ]:
import numpy as np
np.random.seed(42)
W_val = np.random.randn(4, 3)
x_val = np.random.rand(4)
b_val = np.ones(3)

f(x_val, W_val)

In [ ]:
g(x_val, W_val, b_val)

In [ ]:
h(x_val, W_val, b_val)

In [ ]:
i(x_val, W_val, b_val)

# Graph definition and Syntax
## Graph structure

In [ ]:
pydotprint(f, compact=False, outfile='pydotprint_f_notcompact.png')
Image('pydotprint_f_notcompact.png', width=1000)

## Strong typing
### Broadcasting tensors

In [ ]:
r = T.row('r')
print(r.broadcastable)

In [ ]:
c = T.col('c')
print(c.broadcastable)

In [ ]:
f = theano.function([r, c], r + c,allow_input_downcast=True)
print(f([[1, 2, 3]], [[.1], [.2]]))

# Graph Transformations
## Substitution and Cloning
### The `givens` keyword

In [ ]:
x_ = T.vector('x_')
x_n = (x_ - x_.mean()) / x_.std()
f_n = theano.function([x_, W], dot, givens={x: x_n},allow_input_downcast=True)
f_n(x_val, W_val)

### Cloning with replacement

In [ ]:
dot_n, out_n = theano.clone([dot, out], replace={x: (x - x.mean()) / x.std()})                        
f_n = theano.function([x, W], dot_n,allow_input_downcast=True)                                                                  
f_n(x_val, W_val)

## Gradient
### Using `theano.grad`

In [ ]:
y = T.vector('y')
C = ((out - y) ** 2).sum()
dC_dW = theano.grad(C, W)
dC_db = theano.grad(C, b)
# dC_dW, dC_db = theano.grad(C, [W, b])

### Using the gradients

In [ ]:
cost_and_grads = theano.function([x, W, b, y], [C, dC_dW, dC_db],allow_input_downcast=True)
y_val = np.random.uniform(size=3)
print(cost_and_grads(x_val, W_val, b_val, y_val))

In [ ]:
upd_W = W - 0.1 * dC_dW
upd_b = b - 0.1 * dC_db
cost_and_upd = theano.function([x, W, b, y], [C, upd_W, upd_b],allow_input_downcast=True)
print(cost_and_upd(x_val, W_val, b_val, y_val))

In [ ]:
pydotprint(cost_and_upd, outfile='pydotprint_cost_and_upd.png')
Image('pydotprint_cost_and_upd.png', width=1000)

## Shared variables
### Update values

In [ ]:
C_val, dC_dW_val, dC_db_val = cost_and_grads(x_val, W_val, b_val, y_val)
W_val -= 0.1 * dC_dW_val
b_val -= 0.1 * dC_db_val

C_val, W_val, b_val = cost_and_upd(x_val, W_val, b_val, y_val)

### Using shared variables

In [ ]:
x = T.vector('x')
y = T.vector('y')
W = theano.shared(W_val)
b = theano.shared(b_val)
dot = T.dot(x, W)
out = T.nnet.sigmoid(dot + b)
f = theano.function([x], dot,allow_input_downcast=True)  # W is an implicit input
g = theano.function([x], out,allow_input_downcast=True)  # W and b are implicit inputs
print(f(x_val))

In [ ]:
print(g(x_val))

### Updating shared variables

In [ ]:
C = ((out - y) ** 2).sum()
dC_dW, dC_db = theano.grad(C, [W, b])
upd_W = W - 0.1 * dC_dW
upd_b = b - 0.1 * dC_db

cost_and_perform_updates = theano.function(
    inputs=[x, y],
    outputs=C,
    updates=[(W, upd_W),
             (b, upd_b)])

In [ ]:
pydotprint(cost_and_perform_updates, outfile='pydotprint_cost_and_perform_updates.png')
Image('pydotprint_cost_and_perform_updates.png', width=1000)

# Advanced Topics
## Extending Theano
### The easy way: Python

In [ ]:
import theano
import numpy
from theano.compile.ops import as_op

def infer_shape_numpy_dot(node, input_shapes):
    ashp, bshp = input_shapes
    return [ashp[:-1] + bshp[-1:]]

@as_op(itypes=[theano.tensor.fmatrix, theano.tensor.fmatrix],
       otypes=[theano.tensor.fmatrix], infer_shape=infer_shape_numpy_dot)
def numpy_dot(a, b):
   return numpy.dot(a, b)